# FM 5091: Implementation of Black-Scholes Option Pricing in Python
Created on Thu Sep 19 18:17:09 2019 <br>
@author: wrueckl


**Parameters:**
- S: spot price , expressed in domestic currency
- K: strike price, in home currency
- T: tenor (in years)
- r: risk-free interest rate
- sigma: volatility of underlying asset
<br>

#### From Black-Scholes model, know that:
- note: N(d) is used to indicate normal distribution CDF

- Price of underlying lognormally distributed
- C(S,K,r,t,sigma) = SN(d1)-Ke^(-r(T-t))N(d2)
- P(S,K,r,t,sigma) = Ke^(-r(T-t))N(d2)-SN(d1)

- Returns (di) are normally distibuted
- d1 = [ln(S/K)+(r+(sigma^2)/2)T]/sigma*sqrt(T)
- d1 = [ln(S/K)+(r - (sigma^2)/2)T]/sigma*sqrt(T) --> = d1-sigma*sqrt(T)

#### Import packages

In [1]:
import numpy as np
import sympy as sy
import scipy.stats as sp
import math

C:\Users\whitn\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Assign input values

In [2]:
# ----------------------------#
# Benchmark option:
# A call option with the following characteristics should price to $10.89 (use as a check)
#S = 50
#K = 50
#r = 0.05
#T = 1
#sigma = 0.5
# ----------------------------#


# Calc price of call option with the following characteristics:
# Inputs:
S = 60
K = 50
r = 0.05
T = 1
sigma = 0.5

print("Creating Black Scholes lambdas for option with following characteristics: ")
print("spot price = ", S, ", stike price = ", K, ", rate = ", r, ", and tenor = ", T, ", volatility = ", sigma)

Creating Black Scholes lambdas for option with following characteristics: 
spot price =  60 , stike price =  50 , rate =  0.05 , and tenor =  1 , volatility =  0.5


#### Define functions:

Lambda functions to calculate d1 and d2:

In [3]:
# Define anonomyous functions for d1 and d2: ----
d1 = lambda S, K, r, T, sigma:  (np.log(S / K)+(r + 0.5 * sigma ** 2)* T) / (sigma * np.sqrt(T))

d2 = lambda S, K, r, T, sigma: (np.log(S / K)+(r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
print(" ----> d1:", d1(S, K, r, T, sigma))
print(" ----> d2", d2(S, K, r, T, sigma))


 ----> d1: 0.7146431135879092
 ----> d2 0.2146431135879092


#### Define lambda functions to calculate European option prices and greeks:

In [5]:
# Define lamada functions for option prices and greeks: ----

callPrice = lambda S, K, r , T, sigma: (S * sp.norm.cdf(d1(S, K, r ,T, sigma), 0, 1) - K * np.exp(-r * T) * sp.norm.cdf(d2(S, K, r ,T, sigma), 0, 1))
putPrice = lambda S, K, r , T, sigma: K * np.exp(-r * T) * sp.norm.cdf(-d2(S, K, r ,T, sigma), 0, 1) - (S * sp.norm.cdf(-d1(S, K, r ,T, sigma), 0, 1) )

callDelta = lambda d1: sp.norm.cdf(d1)
putDelta = lambda d1: (sp.norm.cdf(d1)-1)

calcGamma = lambda S, sigma, T, d1: (sp.norm.pdf(d1)/(S*sigma*math.sqrt(T)))

calcVega = lambda S, T, d1: (S*sp.norm.pdf(d1)*math.sqrt(T))

callTheta = lambda S, sigma, T, r, K, d1, d2: (-S * sp.norm.pdf(d1) * sigma)/(2 * math.sqrt(T)) - r * K * math.exp(-r * T) * sp.norm.cdf(d2)
putTheta = lambda S, sigma, T, r, K, d1, d2: (-S * sp.norm.pdf(d1) * sigma)/(2 * math.sqrt(T)) +  r * K * math.exp(-r * T) * sp.norm.cdf(-d2)

callRho = lambda K, T, r, d2: K * T * math.exp(-r * T) * sp.norm.cdf(d2)
putRho = lambda K, T, r, d2: -K * T * math.exp(-r * T) * sp.norm.cdf(-d2)

#### Price option and calculate greeks

In [6]:
# Calculate call and put option prices and greeks:-----
call_price = round(callPrice(S, K, r, T, sigma), 4)
put_price = round(putPrice(S, K, r, T, sigma), 4)
print(" ---->  Call price = $", call_price)
print(" ---->  Put price = $", put_price)


call_delta = callDelta(d1(S, K, r ,T, sigma))
put_delta = putDelta(d1(S, K, r ,T, sigma))
print(" ---->  Put delta =", put_delta)
print(" ---->  Call delta =", call_delta)


gamma = round(calcGamma(S, sigma, T, d1(S, K, r ,T, sigma)), 4)
print(" ----> Call and put gamma =", gamma)


vega = round(calcVega(S, T, d1(S, K, r ,T, sigma)), 4)
print(" ----> Call and put vega =", vega)


call_theta = round(callTheta(S, sigma, T, r, K, d1(S, K, r ,T, sigma),  d2(S, K, r ,T, sigma)), 4)
put_theta = round(putTheta(S, sigma, T, r, K, d1(S, K, r ,T, sigma), d2(S, K, r ,T, sigma)), 4)
print(" ----> Call theta =", call_theta)
print(" ----> Put theta =", put_theta)


call_rho = round(callRho(K, T, r, d2(S, K, r ,T, sigma)), 4)
put_rho = round(putRho(K, T, r, d2(S, K, r ,T, sigma)), 4)
print(" ----> Call rho =", call_rho)
print(" ----> Put rho =", put_rho)

 ---->  Call price = $ 17.9327
 ---->  Put price = $ 5.4942
 ---->  Put delta = -0.23741479849765668
 ---->  Call delta = 0.7625852015023433
 ----> Call and put gamma = 0.0103
 ----> Call and put vega = 18.5422
 ----> Call theta = -6.0267
 ----> Put theta = -3.6486
 ----> Call rho = 27.8224
 ----> Put rho = -19.7391
